In [ ]:
import openpyxl
import xlrd
from openpyxl import Workbook
from openpyxl.styles import NamedStyle
from datetime import datetime

# Step 2.1: generating the excel workbook

# Define a custom date format style in openpyxl
date_style = NamedStyle(name="date_style", number_format="DD-MMM-YYYY")

# Open the .xls file using xlrd
xls_file_path = 'Export-Issue-6761.xls'
workbook_xls = xlrd.open_workbook(xls_file_path)

# Create a new .xlsx workbook using openpyxl
workbook_xlsx = Workbook()
sheet_xlsx = workbook_xlsx.active

# Iterate through each sheet in the .xls file
for sheet_index in range(workbook_xls.nsheets):
    sheet_xls = workbook_xls.sheet_by_index(sheet_index)

    # Create a new sheet in the .xlsx workbook for each .xls sheet
    if sheet_index == 0:
        sheet_xlsx.title = sheet_xls.name
    else:
        sheet_xlsx = workbook_xlsx.create_sheet(title=sheet_xls.name)

        # Iterate through each row and column, copying the content to the .xlsx sheet
    for row in range(sheet_xls.nrows):
        for col in range(sheet_xls.ncols):
            cell_value = sheet_xls.cell_value(row, col)
            cell_type = sheet_xls.cell_type(row, col)

            # Check if the cell is a date
            if cell_type == xlrd.XL_CELL_DATE:
                date_value = xlrd.xldate.xldate_as_datetime(cell_value, workbook_xls.datemode)
                cell_xlsx = sheet_xlsx.cell(row=row + 1, column=col + 1, value=date_value)
                cell_xlsx.number_format = date_style.number_format
            else:
                sheet_xlsx.cell(row=row + 1, column=col + 1).value = cell_value

# Save the new .xlsx file
xlsx_file_path = 'Export_Issue_6761.xlsx'
workbook_xlsx.save(xlsx_file_path)


from openpyxl import load_workbook

# Load the workbook and select the issue sheet
workbook = load_workbook('Export_Issue_6761.xlsx')

# Delete File and Definition worksheets
worksheets_to_delete = ['Files', 'Definition']

# Delete the specified worksheets
for sheet_name in worksheets_to_delete:
    if sheet_name in workbook.sheetnames:  # Check if the sheet exists
        sheet = workbook[sheet_name]
        workbook.remove(sheet)

# Save the workbook after deleting the sheets
workbook.save('Export_Issue_6761.xlsx')

ws = workbook.active


# Define the columns to keep (1-based index)
columns_to_keep = [7,8,16,22,25,28,31,34,39,40,42,45,62]

# Create a new workbook and sheet for the output
new_wb = Workbook()
new_ws = new_wb.active
new_ws.title = 'Selected Columns'

# Iterate through rows in the existing sheet and write the selected columns to the new sheet
for row in ws.iter_rows():
    new_row = [row[i-1].value for i in columns_to_keep]
    new_ws.append(new_row)

# Check if the cell is a date as dd-mmm-yyyy

# Define the desired date format style
date_style = NamedStyle(name="date_style", number_format="DD-MMM-YYYY")

# Function to check if a cell's value is a date
def is_date(value):
    if isinstance(value, datetime):
        return True
    return False

# Iterate over all cells in the sheet
for row in new_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

# Save the new workbook with the selected columns
new_wb.save('selected_columns.xlsx')

# Create two new sheets

accepted_ws = new_wb.create_sheet(title='Accepted')
open_ws = new_wb.create_sheet(title='Open')

# Assuming the first row contains headers, copy them to both new sheets
headers = [cell.value for cell in new_ws[1]]
accepted_ws.append(headers)
open_ws.append(headers)

# Iterate through the rows and distribute them based on the 'status' column
status_column = headers.index('Status') + 1  # Assuming the header row is 1-based

for row in new_ws.iter_rows(min_row=2, values_only=True):
    if row[status_column - 1] == 'Accepted':
        accepted_ws.append(row)
    elif row[status_column - 1] == 'Open':
        open_ws.append(row)

# Iterate over all cells in the sheet
for row in accepted_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

# Iterate over all cells in the sheet
for row in open_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

new_wb.save('selected_columns.xlsx')


# Define the columns to in the accepted sheet to keep
new_ws = new_wb['Accepted']
columns_to_keep = ['Identifier','Issue Name','Current Issue Risk Rating','Business Line / Support Function','Review Date','Accepted date','Creation Date']

# Find the indices of the columns to keep based on headers
header_row = [cell.value for cell in new_ws[1]]  # Assuming headers are in the first row
indices_to_keep = [header_row.index(col) + 1 for col in columns_to_keep]

# Create a list to store the rows with only the selected columns
filtered_rows = []

# Copy the header row for the selected columns
selected_headers = [header_row[i - 1] for i in indices_to_keep]
filtered_rows.append(selected_headers)

# Iterate through rows and copy the data for the selected columns
for row in new_ws.iter_rows(min_row=2, values_only=True):  # Start from row 2 to skip headers
    new_row = [row[i - 1] for i in indices_to_keep]
    filtered_rows.append(new_row)

# Clear the original worksheet content
new_ws.delete_rows(1, ws.max_row)

# Write the filtered data back to the original worksheet
for row in filtered_rows:
    new_ws.append(row)

# Iterate over all cells in the sheet
for row in accepted_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

# Save the workbook with the modified worksheet
new_wb.save('selected_columns.xlsx')

# Part one Accepted risk finished


# Part two select High, Medium and low by Current Issue Risk Rating


# Determine Current Issue Risk Ranking

# Extract data from column 3 -Current Issue Risk Rating
from openpyxl import Workbook, load_workbook
import pandas as pd

new_wb = load_workbook('selected_columns.xlsx')
new_ws = new_wb['Open']

# Extract all data from the worksheet
data = []
for row in new_ws.iter_rows(values_only=True):
    data.append(row)

# Define the custom order for sorting
custom_order = {"High": 1, "Medium": 2, "Low": 3}

# Function to get the sorting key
def get_sort_key(value):
    return custom_order.get(value[2], 4)  # Default to 4 if the value is not found

# Sort data based on the third column (index 2)
sorted_data = sorted(data[1:], key=get_sort_key)  # Skip the header row
sorted_data.insert(0, data[0])  # Reinsert the header row at the top

# Clear the existing data
for row in new_ws.iter_rows():
    for cell in row:
        cell.value = None

# Write sorted data back to the worksheet
for i, row in enumerate(sorted_data, start=1):
    for j, value in enumerate(row, start=1):
        new_ws.cell(row=i, column=j, value=value)

# Iterate over all cells in the sheet
for row in open_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

new_wb.save('selected_columns.xlsx')



# Create two new sheets -high findings or medium/low findings

High_ws = new_wb.create_sheet(title='High')
Medium_low_ws = new_wb.create_sheet(title='Medium_Low')

# Assuming the first row contains headers, copy them to both new sheets
headers = [cell.value for cell in new_ws[1]]
High_ws.append(headers)
Medium_low_ws.append(headers)

# Iterate through the rows and distribute them based on the 'Current Issue Risk Rating' column
status_column = headers.index('Current Issue Risk Rating') + 1  # Assuming the header row is 1-based
for row in new_ws.iter_rows(min_row=2, values_only=True):
    if row[status_column - 1] == 'High':
        High_ws.append(row)
    elif row[status_column - 1] in ['Medium', 'Low']:
        Medium_low_ws.append(row)

# Iterate over all cells in the sheet
for row in High_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

# Iterate over all cells in the sheet
for row in Medium_low_ws.iter_rows():
    for cell in row:
        if is_date(cell.value):
            # Check the current cell's number format
            if cell.number_format != "DD-MMM-YYYY":
                # Convert the cell's format to the desired format
                cell.number_format = date_style.number_format

new_wb.save('selected_columns.xlsx')

# only keep the useful columns for high and medium worksheets

sheet1 = new_wb['High']
sheet2 = new_wb['Medium_Low']

# Define the columns to keep for each sheet

columns_to_keep_sheet1 = ['A', 'B', 'C', 'E', 'F', 'I', 'J', 'K', 'L', 'M']
columns_to_keep_sheet2 = ['A', 'B', 'C', 'D', 'E', 'F', 'I', 'J', 'K', 'L', 'M']

# Remove unwanted columns from the worksheet

# Convert column letters to indices
columns_to_keep_sheet1 = [openpyxl.utils.column_index_from_string(col) for col in columns_to_keep_sheet1]
columns_to_keep_sheet2 = [openpyxl.utils.column_index_from_string(col) for col in columns_to_keep_sheet2]

# Delete the unwanted columns from Sheet1
for col in range(sheet1.max_column, 0, -1):  # Loop backwards to avoid shifting issues
    if col not in columns_to_keep_sheet1:
        sheet1.delete_cols(col)

# Delete the unwanted columns from Sheet2
for col in range(sheet2.max_column, 0, -1):  # Loop backwards to avoid shifting issues
    if col not in columns_to_keep_sheet2:
        sheet2.delete_cols(col)


# arrange the column-creation date
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string

# Define the columns to move and the target positions
column_to_move_sheet1 = 'J'  # Column to move in Sheet1
column_to_move_sheet2 = 'K'  # Column to move in Sheet2
target_position_sheet1 = 6   # Target position in Sheet1
target_position_sheet2 = 7   # Target position in Sheet2

# Helper function to move a column within a worksheet
def move_column(sheet, column_letter, target_position):
    # Extract data from the column
    col_idx = column_index_from_string(column_letter)
    column_data = [cell[0] for cell in sheet.iter_rows(min_col=col_idx, max_col=col_idx, values_only=True)]

    # Delete the original column
    sheet.delete_cols(col_idx)

    # Insert the column at the target position
    sheet.insert_cols(target_position)
    for row_idx, value in enumerate(column_data, start=1):
        sheet.cell(row=row_idx, column=target_position, value=value)

# Move the specified columns in the respective sheets
move_column(sheet1, column_to_move_sheet1, target_position_sheet1)
move_column(sheet2, column_to_move_sheet2, target_position_sheet2)

# Save the workbook after modifying the sheets
new_wb.save('selected_columns_V3.xlsx')


# Step 2.2: create the docx file from Issue Description,Recommendation,Status Update
import pandas as pd
from docxtpl import DocxTemplate

# Load the template
template_path = 'template.docx'

# Define the path to the Excel file
excel_file_path = 'selected_columns_V3.xlsx'

# Read the 'High' and 'Medium_Low' worksheets from the Excel workbook
sheet1_df = pd.read_excel(excel_file_path, sheet_name='High')
sheet2_df = pd.read_excel(excel_file_path, sheet_name='Medium_Low')

# Define a function to create a docx file from a row
def create_docx_from_row(row, index, prefix):
    # Define the context for the current row
    context = {
        'Issue_Description': row['Issue Description'],
        'Recommendation': row['Recommendation'],
        'Status_Update': row['Status Update']
    }

    # Create a new DocxTemplate object for each iteration to avoid overwriting
    doc = DocxTemplate(template_path)

    # Render the template with the current context
    doc.render(context)

    # Save the document with a unique filename
    doc.save(f'{prefix}_generated_doc_{index + 1}.docx')

# Iterate over the rows in the 'High' DataFrame and create a docx file for each
for index, row in sheet1_df.iterrows():
    create_docx_from_row(row, index, 'High')

# Iterate over the rows in the 'Medium_Low' DataFrame and create a docx file for each
for index, row in sheet2_df.iterrows():
    create_docx_from_row(row, index, 'Medium_Low')


# Delete the two columns - 'Issue Description', 'Recommendation', 'Status Update'
# Clean column -  'Issue Description'

import pandas as pd

# Define the path to the Excel file
excel_file_path = 'selected_columns_V3.xlsx'
output_file_path = 'selected_columns_V3.xlsx'

# Read the three worksheets from the Excel workbook
sheet_high_df = pd.read_excel(excel_file_path, sheet_name='High')
sheet_medium_low_df = pd.read_excel(excel_file_path, sheet_name='Medium_Low')
sheet_accepted_df = pd.read_excel(excel_file_path, sheet_name='Accepted')

# Define the columns to delete
columns_to_delete = ['Issue Description', 'Recommendation', 'Status Update']

# Drop the specified columns from 'High' and 'Medium_Low' worksheets
sheet_high_df = sheet_high_df.drop(columns=columns_to_delete)
sheet_medium_low_df = sheet_medium_low_df.drop(columns=columns_to_delete)

# Create a new blank column 'Issue Description' in both DataFrames
sheet_high_df['Issue Description'] = ''
sheet_medium_low_df['Issue Description'] = ''

# Save the modified DataFrames back to an Excel file
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    sheet_high_df.to_excel(writer, sheet_name='High', index=False)
    sheet_medium_low_df.to_excel(writer, sheet_name='Medium_Low', index=False)
    sheet_accepted_df.to_excel(writer, sheet_name='Accepted', index=False)



# Define the date format
import pandas as pd

# Define the path to the Excel file
excel_file_path = 'selected_columns_V3.xlsx'
output_file_path = 'selected_columns_V3.xlsx'

# Read the worksheets from the Excel workbook
sheet_high_df = pd.read_excel(excel_file_path, sheet_name='High')
sheet_medium_low_df = pd.read_excel(excel_file_path, sheet_name='Medium_Low')
sheet_accepted_df = pd.read_excel(excel_file_path, sheet_name='Accepted')

# Function to identify and format date columns
def format_date_columns(df):
    for column in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[column]):
            # If already datetime, format the dates
            df[column] = df[column].dt.strftime('%d-%b-%Y')
        elif pd.api.types.is_object_dtype(df[column]):
            try:
                # Attempt to convert to datetime and then format
                df[column] = pd.to_datetime(df[column], errors='raise')
                if pd.api.types.is_datetime64_any_dtype(df[column]):
                    df[column] = df[column].dt.strftime('%d-%b-%Y')
            except (ValueError, TypeError):
                # If conversion fails, skip formatting
                pass

# Apply the date formatting function to each DataFrame
format_date_columns(sheet_high_df)
format_date_columns(sheet_medium_low_df)
format_date_columns(sheet_accepted_df)

# Save the formatted DataFrames back to a new Excel file
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    sheet_high_df.to_excel(writer, sheet_name='High', index=False)
    sheet_medium_low_df.to_excel(writer, sheet_name='Medium_Low', index=False)
    sheet_accepted_df.to_excel(writer, sheet_name='Accepted', index=False)


# format

from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from openpyxl.styles import Alignment

# Define the border style
orange_border = Border(
    left=Side(style='thin', color='FFA500'),
    right=Side(style='thin', color='FFA500'),
    top=Side(style='thin', color='FFA500'),
    bottom=Side(style='thin', color='FFA500')
)

# Load the workbook to be modified
input_file_path = 'selected_columns_V3.xlsx'
output_file_path = 'selected_columns_V6.xlsx'
workbook = load_workbook(input_file_path)

# Define the border style (thin in this example) and color (orange)
orange_border = Border(
    left=Side(border_style="thin", color="FFA500"),
    right=Side(border_style="thin", color="FFA500"),
    top=Side(border_style="thin", color="FFA500"),
    bottom=Side(border_style="thin", color="FFA500")
)

# Apply the border style to all cells in all sheets
from openpyxl.styles import Alignment
for sheet in workbook.worksheets:
    for row in sheet.iter_rows():
        for cell in row:
            cell.border = orange_border

from openpyxl.styles import Font
# Define the font style (name, size, color, etc.)
font_style = Font(name="ING ME (body)", size=8, color="000000")  # Black color

# Apply the font style to all cells in the worksheet
for sheet in workbook.worksheets:
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = font_style


# Adjust column widths:
# Iterate over each sheet in the workbook
for sheet in workbook.worksheets:
    # Adjust column widths and wrap text
    for column_cells in sheet.columns:
        max_length = 0
        column = column_cells[0].column_letter  # Get the column letter
        for cell in column_cells:
            # Wrap text in each cell
            cell.alignment = Alignment(vertical='top', wrap_text=True)

            # Determine the maximum length of the cell content in the column
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass

        # Set the column width, adding a little extra space
    sheet.column_dimensions[column].width = max_length

# Save the workbook with the new styles
workbook.save(output_file_path)